<b>Exercise 9: Simulation of a small (N=50) network of single-compartmental neurons.</b><br><br>
The neurons are connected by current-based synapses as in the previous exercise. Each pair of neurons is connected at a fixed connection probability. In addition, a given number of neurons is initially ($t$ = 100ms) stimulated with a square-pulse current.

In [1]:
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [8]:
### Run NEURON with Python interface
def runneuron(N=50,isadaptive=True,amp=0.12,dur=10,Nstim=1,pconn=0.3,amp2=1.0,tau1=0.5,tau2=20.0,glsoma=0.03,cm=1.0,rdseed=1):
    my_start_time = time.time()
    numpy.random.seed(rdseed)
    connM = (numpy.random.rand(N,N) < pconn)*(1-numpy.eye(N)) #Make connectivity matrix, don't allow autapses (self-connections)
        
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library

    h('objref cvode')                                #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h('create soma['+str(N)+']')                     #Create soma for N neurons

    h('access soma[0]')                              #Make soma[0] the currently accessed section

    for i in range(0,N):
        h.soma[i].insert('hh')                       #Insert the Hodgkin-Huxley mechanism (includes leak) to soma
        h.soma[i].nseg = 1                                  

    #Set the passive parameters for each section:
    for i in range(0,N):
        h.soma[i].cm = cm;
        h.soma[i].diam = 10;
        h.soma[i].L = 20;
        h.soma[i].gl_hh = 0.001*glsoma

    h.dt = 0.025                                     #Set the time step to 0.025 ms
    h.tstop = 300                                    #Continue the simulation until 200 ms

    h('objref stims['+str(Nstim)+'], syns')          #Declare stimulation objects and synapse objects
    for i in range(0,Nstim):
        h.stims[i] = h.IClamp(0.5, sec = h.soma[i])  #Make stims[i] an IClamp object, stimulating center (0.5) of soma
        h.stims[i].delay = 100                       #Stimulation starts at 100 ms
        h.stims[i].dur = dur                         #and lasts 10 ms (by default)
        h.stims[i].amp = amp                         #and has a given amplitude (default 0.12 nA)

    h('objref nc, ncs, fih, Vrecs, trec')            #Declare objects nc, nilstim, fih, Vrecs and trec
    nSyn = 0
    h.ncs = h.List()
    h.syns = h.List()
    Vrecs = []
    h('objref spikes, spikedCells, nil')
    h.spikes = h.Vector()
    h.spikedCells = h.Vector()
    for i in range(0,N):
        for j in range(0,N):
            if connM[i,j]:
                h('{soma['+str(j)+'] syns.append(new Exp2Syn(0.5))}')
                h.syns[nSyn].tau1 = tau1             #Synaptic stimulation has given rise and
                h.syns[nSyn].tau2 = tau2             #decay time constants
                h('{soma['+str(i)+'] ncs.append(new NetCon(&v(0.5), syns.o['+str(nSyn)+']))}') #Insert a NetCon object that activates syns[nSyn] whenever soma.v(0.5) crosses a threshold
                h.ncs[nSyn].threshold = -30                             #set threshold to -30 mV
                h('{ncs.o['+str(nSyn)+'].weight = '+str(0.001*amp2)+'}')#Maximal synaptic conductance amp (default 1 nS (0.001 uS)) 
                nSyn = nSyn + 1

        Vrecs.append(h.Vector())                                #Record somatic membrane potentials and time
        Vrecs[i].record(h.soma[i](0.5)._ref_v, sec=h.soma[i])
        h('{soma['+str(i)+'] nc = new NetCon(&v(0.5),nil)}')
        h.nc.threshold = -30
        h('{nc.record(spikes, spikedCells)}')
        
    trec = h.Vector()
    trec.record(h._ref_t)
    
    
    
    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations

    plt.close("all")
    f, axs = plt.subplots(1, 2, sharey=False)
    for i in range(0,N):
        axs[0].plot(trec,20*i+numpy.array(Vrecs[i]))
    axs[1].plot(numpy.array(h.spikes), numpy.array(h.spikedCells),'b.')
    #axs[1].plot(trec,Vrec2)
    #for i in range(0,2):
    #    axs[i].set_title('soma-'+str(i+1))
    #    axs[i].set_xlim([0, 300])
    #    axs[i].set_ylim([-80, 40])
    #    axs[i].set_ylabel('$V_m(t)$ (mV)')
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()



In [9]:
slider = widgets.interact(runneuron, N=fixed(50), isadaptive=fixed(True),
                          amp=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.12,description='Amplitude of stimulus: amp',layout=Layout(width='50%'), continuous_update = False), 
                          dur=widgets.FloatSlider(min=0.0,max=100.0,step=1.0,value=10,description='Duration of stimulus: dur',layout=Layout(width='50%'), continuous_update = False), 
                          Nstim=widgets.IntSlider(min=0,max=10,step=1,value=1,description='Number of stimulated cells: Nstim',layout=Layout(width='50%'), continuous_update = False), 
                          pconn=widgets.FloatSlider(min=0.0,max=1.0,step=0.05,value=0.3,description='Connection probability: pconn',layout=Layout(width='50%'), continuous_update = False),
                          amp2=widgets.FloatSlider(min=0.0,max=1.0,step=0.02,value=0.1,description='Synaptic weight: amp2',layout=Layout(width='50%'), continuous_update = False), 
                          tau1=widgets.FloatSlider(min=0.1,max=15.0,step=0.1,value=0.5,description='Synaptic rise time const: tau1',layout=Layout(width='50%'), continuous_update = False),
                          tau2=widgets.FloatSlider(min=15.0,max=100.0,step=1,value=20,description='Synaptic decay time const: tau2',layout=Layout(width='50%'), continuous_update = False),
                          glsoma=widgets.FloatSlider(min=0.0,max=0.5,step=0.01,value=0.03,description='Leak conductance, soma: glsoma',layout=Layout(width='50%'), continuous_update = False), 
                          cm=fixed(1.0), 
                          rdseed=widgets.IntSlider(min=1,max=20,step=1,value=1,description='Random number seed: rdseed',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(FloatSlider(value=0.12, continuous_update=False, description='Amplitude of stimulus: amp…

#Question 1: Increase the synaptic weight (amp2) and/or the number of current-clamp-stimulated neurons (Nstim) to make the whole network fire until 300 ms. Would the activity continue until arbitrary time if tstop was increased?

#Question 2: Can you make the firing activity spread throughout most of the neurons but then stop? <i>Hint: use a small connection probability (0.05 or 0.1)</i>. Is the phenomenon stable (try with different random number seeds)?